In [ ]:
!pip install -q flwr[simulation] flwr-datasets[vision] torch torchvision matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import torch
from torch import nn
from torch import optim
import os
import numpy as np


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import TensorDataset, DataLoader, Subset

import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Metrics, Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
disable_progress_bar()

Training on cpu
Flower 1.15.2 / PyTorch 2.5.1+cu124


In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def carregar_dadosv2(train_data, test_data, test_size=0.2, random_state=42):
    # Load data from CSV file
    scaler = StandardScaler()

    train_data = pd.read_csv(train_data)
    val_data = pd.read_csv(test_data)

    # Split data into features (X) and targets (y) ("hysteresis")
    X = train_data.drop(['hysteresis', 'joule'], axis=1)
    y = train_data[['hysteresis', 'joule']]

    # Perform train-test split
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=random_state)

    X_test = val_data.drop(['hysteresis', 'joule'], axis=1)
    y_test = val_data[['hysteresis', 'joule']]

    X_train = scaler.fit_transform(X_train.values)
    X_val = scaler.transform(X_val.values)
    X_test = scaler.transform(X_test.values)

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)


    return X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor

In [ ]:
import matplotlib.pyplot as plt
import torch

def plot_comparacao(y_true, y_pred):
    # Converta para numpy se estiverem em tensores
    if torch.is_tensor(y_true):
        y_true = y_true.cpu().numpy()
    if torch.is_tensor(y_pred):
        y_pred = y_pred.cpu().numpy()

    # Calcular métricas de avaliação
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)

    # Plotar a comparação
    plt.figure(figsize=(10, 8))
    plt.scatter(y_true, y_pred, color='blue', alpha=0.5, s=50, label='Previsões')

    # Linha de referência (Y = X)
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--', linewidth=2, label='Linha de Referência')

    # Adicionar título e legendas
    plt.xlabel('Valores Reais', fontsize=12)
    plt.ylabel('Valores Previstos', fontsize=12)
    plt.title('Comparação entre Valores Reais e Previstos', fontsize=14)
    plt.legend()

    # Exibir métricas no gráfico
    plt.text(min_val, max_val * 0.95, f'MAE: {mae:.4f}', fontsize=12, color='green')
    plt.text(min_val, max_val * 0.90, f'MSE: {mse:.4f}', fontsize=12, color='green')
    plt.text(min_val, max_val * 0.85, f'RMSE: {rmse:.4f}', fontsize=12, color='green')

    # Adicionar uma grade
    plt.grid(True, linestyle='--', alpha=0.7)

    # Exibir o gráfico
    plt.show()


In [ ]:
class Net(nn.Module):
    def __init__(self, input_dim, output_dim, num_layers, hidden_dim):
        super(Net, self).__init__()
        layers = []

        # Construindo as camadas com base no número de camadas
        for i in range(num_layers):
            layers.append(nn.Linear(input_dim if i == 0 else hidden_dim, hidden_dim))
            layers.append(nn.ReLU())

        # Guardar o índice da penúltima camada
        self.penultimate_layer_index = len(layers) - 2

        # Adicionar a camada final de saída
        layers.append(nn.Linear(hidden_dim, output_dim))

        # Definir a sequência de camadas como parte da rede
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        penultimate_output = None

        # Executar o forward manualmente para capturar a penúltima camada
        for i, layer in enumerate(self.network):
            x = layer(x)
            if i == self.penultimate_layer_index:
                penultimate_output = x  # Armazenar a saída da penúltima camada

        # Retornar a saída final e a penúltima camada
        return x, penultimate_output

In [ ]:
from torch import tensor

from sklearn.metrics import mean_squared_error, mean_absolute_error
from torch.optim.lr_scheduler import ReduceLROnPlateau


def load_model(filename, model_class, input_size, output_size):
    print("Carregando modelo...")
    checkpoint = torch.load(filename)

    model = model_class(input_size, output_size)

    model.load_state_dict(checkpoint['state_dict'])

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    optimizer.load_state_dict(checkpoint['optimizer'])

    return model, optimizer

def train(model, num_epochs, optimizer, train_loader, val_loader):
    model = model.to(device)
    criterion = nn.L1Loss().to(device)  # Loss function

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0  # Track training loss

        for train_set, target in train_loader:
            # Move the inputs and targets to the appropriate device
            train_set = train_set.to(device)
            target = target.to(device)

            optimizer.zero_grad()  # Clear gradients

            # Forward pass
            outputs, _ = model(train_set)
            loss = criterion(outputs, target)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        if (epoch + 1) % 100 == 0:
            avg_train_loss = running_loss / len(train_loader)
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_train_loss:.4f}')

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for val_set, val_target in val_loader:
                val_set = val_set.to(device)
                val_target = val_target.to(device)

                val_outputs, _ = model(val_set)
                loss = criterion(val_outputs, val_target)
                val_loss += loss.item()

            if (epoch + 1) % 100 == 0:
                avg_val_loss = val_loss / len(val_loader)
                print(f'Epoch [{epoch+1}/{num_epochs}], Val_Loss: {avg_val_loss:.4f}')

def save_model(state,filename="Saved_Model.pth"):
  print("salvando modelo...")
  torch.save(state,filename)



def test(net, testloader):
    """Evaluate the network on the entire test set for a regression problem."""
    criterion = torch.nn.MSELoss()  # Use MSE as the loss
    total_loss, total_mse = 0.0, 0.0
    net.eval()

    with torch.no_grad():
        for batch in testloader:
            images, targets = batch[0].to(DEVICE), batch[1].to(DEVICE)
            outputs , _ = net(images)

            # Calculate the loss (MSE)
            loss = criterion(outputs, targets)
            total_loss += loss.item()

            # Calculate the MSE for accuracy
            mse = torch.mean((outputs - targets) ** 2)
            total_mse += mse.item()

    avg_loss = total_loss / len(testloader.dataset)
    avg_mse = total_mse / len(testloader)  # Average per batch for mse

    return avg_loss, avg_mse

In [ ]:
def change_dataset(folder_path):
    folder_path = folder_path
    os.chdir(folder_path)

    X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor = carregar_dadosv2('dados_de_treino.csv','dados_de_teste.csv')

    X_train_tensor = X_train_tensor.to(device)
    y_train_tensor = y_train_tensor.to(device)
    X_val_tensor = X_val_tensor.to(device)
    y_val_tensor = y_val_tensor.to(device)
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)

    histeresis_train_tensor = [pair[0].item() for pair in y_train_tensor]
    histeresis_train_tensor = torch.tensor(histeresis_train_tensor, dtype=torch.float32).to(device)
    histeresis_train_tensor = histeresis_train_tensor.unsqueeze(1)

    joule_train_tensor = [pair[1].item() for pair in y_train_tensor]
    joule_train_tensor = torch.tensor(joule_train_tensor, dtype=torch.float32).to(device)
    joule_train_tensor = joule_train_tensor.unsqueeze(1)

    histeresis_val_tensor = [pair[0].item() for pair in y_val_tensor]
    histeresis_val_tensor = torch.tensor(histeresis_val_tensor, dtype=torch.float32).to(device)
    histeresis_val_tensor = histeresis_val_tensor.unsqueeze(1)

    joule_val_tensor = [pair[1].item() for pair in y_val_tensor]
    joule_val_tensor = torch.tensor(joule_val_tensor, dtype=torch.float32).to(device)
    joule_val_tensor = joule_val_tensor.unsqueeze(1)

    histeresis_test_tensor = [pair[0].item() for pair in y_test_tensor]
    histeresis_test_tensor = torch.tensor(histeresis_test_tensor, dtype=torch.float32).to(device)
    histeresis_test_tensor = histeresis_test_tensor.unsqueeze(1)

    joule_test_tensor = [pair[1].item() for pair in y_test_tensor]
    joule_test_tensor = torch.tensor(joule_test_tensor, dtype=torch.float32).to(device)
    joule_test_tensor = joule_test_tensor.unsqueeze(1)

    return X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor, histeresis_train_tensor, joule_train_tensor, histeresis_val_tensor, joule_val_tensor, histeresis_test_tensor, joule_test_tensor

In [ ]:
X_train_tensorn, y_train_tensorn, X_val_tensorn, y_val_tensorn, X_test_tensorn, y_test_tensorn, histeresis_train_tensorn, joule_train_tensorn, histeresis_val_tensorn, joule_val_tensorn, histeresis_test_tensorn, joule_test_tensorn = change_dataset('/content/drive/MyDrive/FederatedLearning-main/Data/IPMSM_datasets/dataset_for_iron_losses_of_IPMSMs/Nabla')
X_train_tensor2d, y_train_tensor2d, X_val_tensor2d, y_val_tensor2d, X_test_tensor2d, y_test_tensor2d, histeresis_train_tensor2d, joule_train_tensor2d, histeresis_val_tensor2d, joule_val_tensor2d, histeresis_test_tensor2d, joule_test_tensor2d = change_dataset('/content/drive/MyDrive/FederatedLearning-main/Data/IPMSM_datasets/dataset_for_iron_losses_of_IPMSMs/2D')
X_train_tensorv, y_train_tensorv, X_val_tensorv, y_val_tensorv, X_test_tensorv, y_test_tensorv, histeresis_train_tensorv, joule_train_tensorv, histeresis_val_tensorv, joule_val_tensorv, histeresis_test_tensorv, joule_test_tensorv = change_dataset('/content/drive/MyDrive/FederatedLearning-main/Data/IPMSM_datasets/dataset_for_iron_losses_of_IPMSMs/V')

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 512

def split_tensor_dataset(tensor_data):
    subset_size = len(tensor_data) // NUM_CLIENTS
    indices = np.random.permutation(len(tensor_data))
    subsets = []
    for i in range(NUM_CLIENTS):
        start = i * subset_size
        end = (i + 1) * subset_size if i != NUM_CLIENTS - 1 else len(tensor_data)
        subset_indices = indices[start:end]
        subsets.append(Subset(tensor_data, subset_indices))
    return subsets

def load_datasets(partition_id: int):
    # Converte tensores para TensorDatasets

    train_dataset = TensorDataset(X_train_tensorv, histeresis_train_tensorv)
    val_dataset = TensorDataset(X_val_tensorv, histeresis_val_tensorv)
    test_dataset = TensorDataset(X_test_tensorv, histeresis_test_tensorv)

    # Divide o dataset entre os clientes
    train_subsets = split_tensor_dataset(train_dataset)
    val_subsets = split_tensor_dataset(val_dataset)

    # Retorna o DataLoader específico para o cliente com base no partition_id
    train_loader = DataLoader(train_subsets[partition_id], batch_size=32, shuffle=True)
    val_loader = DataLoader(val_subsets[partition_id], batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    return train_loader, val_loader, test_loader

In [ ]:
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [ ]:
class FlowerClient(NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)


    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, 10, optim.Adam(self.net.parameters()),self.trainloader,self.valloader)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

In [ ]:
def client_fn(context: Context) -> Client:
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net(14, 1, 16, 40).to(DEVICE)

    # Get data partition ID from node_config
    partition_id = context.node_config["partition-id"]

    # Ensure the dataset loading function supports partition_id, or adjust it accordingly
    try:
        trainloader, valloader, _ = load_datasets(partition_id=partition_id)
    except TypeError:
        print(f"load_datasets() does not support partition_id, modifying call.")
        trainloader, valloader, _ = load_datasets()  # Adjust to match your actual function signature

    # Create a Flower client representing the organization
    return FlowerClient(net, trainloader, valloader).to_client()

# Create the ClientApp
client = ClientApp(client_fn=client_fn)

In [ ]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE.type == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

In [ ]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [ ]:
from typing import Dict, Tuple, Optional
from sklearn.metrics import mean_squared_error, mean_absolute_error
def evaluate(server_round: int, parameters: flwr.common.NDArrays, config: Dict[str, float]) -> Optional[Tuple[float, Dict[str, float]]]:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Criar o modelo e transferi-lo para o dispositivo
    net = Net(14, 1, 16, 40).to(DEVICE)

    # Carregar o DataLoader para os dados de teste
    _, _, testloader = load_datasets(0)  # Ajuste conforme seu código para carregar dados

    # Atualizar o modelo com os parâmetros mais recentes
    set_parameters(net, parameters)

    # Inicializar listas para armazenar previsões e rótulos
    all_preds = []
    all_labels = []

    net.eval()  # Colocar o modelo em modo de avaliação
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs,_ = net(inputs)
            all_preds.append(outputs.cpu().numpy())  # Armazena as previsões
            all_labels.append(labels.cpu().numpy())  # Armazena os valores reais

    # Concatena as previsões e labels em um único array
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Cálculo das métricas
    mse = mean_squared_error(all_labels, all_preds)
    mae = mean_absolute_error(all_labels, all_preds)
    mape = np.mean(np.abs((all_labels - all_preds) / all_labels)) * 100

    # Exibe os resultados
    print(f"Round {server_round} - MSE: {mse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%")

    return mse, {"MAE": mae, "MAPE": mape}  # Retornando MSE e um dicionário com MAE e MAPE

In [ ]:
def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behaviour.

    You can use settings in `context.run_config` to parameterize the
    construction of all elements (e.g the strategy or the number of rounds)
    wrapped in the returned ServerAppComponents object.
    """

    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=2,
        min_evaluate_clients=2,
        min_available_clients=2,
        evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
        evaluate_fn=evaluate  # Função de avaliação no servidor
    )

    # Configure the server for 5 rounds of training
    config = ServerConfig(num_rounds=50)

    return ServerAppComponents(strategy=strategy, config=config)


# Create a new server instance with the updated FedAvg strategy
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=50, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=14444) 2025-03-06 17:39:05.376118: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=14445) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=14445) E0000 00:00:1741282745.666210   14445 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=14445) E0000 00:00:1741282745.818576   14445 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO :      Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters
IN

Round 0 - MSE: 1.0159, MAE: 0.8426, MAPE: 108.57%


INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      fit progress: (1, 0.29173532128334045, {'MAE': 0.368645042181015, 'MAPE': 111.04252338409424}, 28.762029156000153)
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)


Round 1 - MSE: 0.2917, MAE: 0.3686, MAPE: 111.04%


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)
INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      fit progress: (2, 0.006222297437489033, {'MAE': 0.056147415190935135, 'MAPE': 42.26296544075012}, 55.614614040000106)
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)


...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 49]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)
INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      fit progress: (49, 0.0007351883687078953, {'MAE': 0.018832368776202202, 'MAPE': 19.81159597635269}, 1286.7476662640001)
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)


Round 49 - MSE: 0.0007, MAE: 0.0188, MAPE: 19.81%


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 50]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)
INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      fit progress: (50, 0.000703586672898382, {'MAE': 0.018458940088748932, 'MAPE': 18.039540946483612}, 1313.240080179)
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)


Round 50 - MSE: 0.0007, MAE: 0.0185, MAPE: 18.04%


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 50 round(s) in 1313.55s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.008073824553055687
INFO :      		round 2: 0.00021433106279146436
INFO :      		round 3: 0.00016608453747865402
INFO :      		round 4: 0.00011965454147506587
INFO :      		round 5: 8.74840356692971e-05
INFO :      		round 6: 9.114183312977758e-05
INFO :      		round 7: 7.552059895984488e-05
INFO :      		round 8: 7.095841430253054e-05
INFO :      		round 9: 7.249831815103167e-05
INFO :      		round 10: 6.0364367252283475e-05
INFO :      		round 11: 5.742833860835744e-05
INFO :      		round 12: 5.4396267288464744e-05
INFO :      		round 13: 5.1782954729309214e-05
INFO :      		round 14: 5.163860553875565e-05
INFO :      		round 15: 4.441307287195419e-05
INFO :      		round 16: 4.961599510065035e-05
INFO :      		round 17: 4.412384989279272e-05
INFO :      		round 18: 4.97212